In [177]:
import librosa
import numpy as np
import os
import random
import tensorflow as tf

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
from keras import Input

In [203]:
DATASET_DIR = '/home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets.'
SAMPLE_RATE = 16000
WINDOW_SEC = 5          # 1 second windows
HOP_SEC = 1         # 50% overlap
N_MFCC = 10
print(os.path.exists(DATASET_DIR))

True


In [215]:
X = []
y = []

window_samples = int(WINDOW_SEC * SAMPLE_RATE)
hop_samples = int(HOP_SEC * SAMPLE_RATE)

In [216]:
for root, dirs, files in os.walk(DATASET_DIR):
    # print(root)
    # print(dirs)
    # print(files)
    for file in files:
        if file.lower().endswith(".wav"):
            file_path = os.path.join(root, file)

            if 'not_axe' in root:
                label = 1
            else:
                label = 0
            print("Audio file:", file_path)
            print("Label:", label)

            audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)

            for start in range(0, len(audio) - window_samples, hop_samples):
                window = audio[start:start + window_samples]

            # MFCC extraction
                mfcc = librosa.feature.mfcc(
                y=window,
                sr=sr,
                n_mfcc=N_MFCC
            )

            mfcc_mean = np.mean(mfcc, axis=1)

            X.append(mfcc_mean)
            y.append(label)

Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/410768__selector__axe-chopping-in-the-forest_part-04.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/23709__hazure__timber_part-01.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/367102__anandthethird__wood-chopping-ambience_part-03.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/366368__piggimon__wood-variations_part-02.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/410768__selector__axe-chopping-in-the-forest_part-06.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-datasets./axe/410768__selector__axe-chopping-in-the-forest_part-10.wav
Label: 0
Audio file: /home/dip__4/Documents/datasets-not-for-college/binary-classified-dataset

In [217]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int8)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Axe samples:", np.sum(y == 0))
print("Not-axe samples:", np.sum(y == 1))

X shape: (63, 10)
y shape: (63,)
Axe samples: 33
Not-axe samples: 30


In [218]:
print(X[0].shape)          # should be (N_MFCC,)
print(np.isnan(X).any())  # should be False

(10,)
False


In [219]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [220]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))

[]


In [221]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(8, activation='relu', input_shape=(10,)),
    # Dense(8, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [222]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [223]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [213]:
history = model.fit(
    X,
    y,
    epochs=15,
    batch_size=8,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/15
7/7 [==============================] - 0s 12ms/step - loss: 1.1488 - accuracy: 0.3800 - val_loss: 1.1966 - val_accuracy: 0.0000e+00
Epoch 2/15
7/7 [==============================] - 0s 3ms/step - loss: 1.1176 - accuracy: 0.3800 - val_loss: 1.1763 - val_accuracy: 0.0000e+00
Epoch 3/15
7/7 [==============================] - 0s 3ms/step - loss: 1.0867 - accuracy: 0.3800 - val_loss: 1.1605 - val_accuracy: 0.0000e+00
Epoch 4/15
7/7 [==============================] - 0s 4ms/step - loss: 1.0524 - accuracy: 0.3800 - val_loss: 1.1407 - val_accuracy: 0.0000e+00
Epoch 5/15
7/7 [==============================] - 0s 3ms/step - loss: 1.0228 - accuracy: 0.3800 - val_loss: 1.1233 - val_accuracy: 0.0000e+00
Epoch 6/15
7/7 [==============================] - 0s 3ms/step - loss: 0.9911 - accuracy: 0.3800 - val_loss: 1.1058 - val_accuracy: 0.0000e+00
Epoch 7/15
7/7 [==============================] - 0s 3ms/step - loss: 0.9618 - accuracy: 0.3800 - val_loss: 1.0865 - val_accuracy: 0.0000e+00
Epoch

In [214]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 8)                 88        
                                                                 
 dense_28 (Dense)            (None, 1)                 9         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [190]:
pred = model.predict(X[40:56])
print(pred)
for pre in pred:
    if pre > 0.7:
        print("AXE detected")
    else:
        print('Not AXE detected')

1/1 [==============================] - 0s 26ms/step
[[0.7289389 ]
 [0.42924067]
 [0.4216908 ]
 [0.48496872]
 [0.45039114]
 [0.43986005]
 [0.29159537]
 [0.4721206 ]
 [0.41999397]
 [0.4309449 ]
 [0.49727046]
 [0.33775648]
 [0.4543964 ]
 [0.41251174]
 [0.42378324]
 [0.23421246]]
AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected
Not AXE detected


In [191]:
# model.save('idk_about_this.keras')

In [192]:
# model.save('68,789.keras')

In [193]:
# model = tf.keras.models.load_model('68,789.keras')

In [194]:
if model: print(True)

True


In [195]:
def representative_data_generation():
    for i in range(len(X)):
        yield [X[i].astype(np.float32)]

In [196]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide representative dataset
converter.representative_dataset = representative_data_generation

# Force full INT8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4yrxphdl/assets


INFO:tensorflow:Assets written to: /tmp/tmp4yrxphdl/assets
/home/dip__4/miniconda3/envs/py310/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-12-24 01:54:33.898435: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-12-24 01:54:33.898448: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-12-24 01:54:33.898549: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4yrxphdl
2025-12-24 01:54:33.898939: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-12-24 01:54:33.898944: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4yrxphdl
2025-12-24 01:54:33.899999: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-12

In [197]:
# with open('weights.tflite', 'wb') as f:
#     f.write(tflite_model)